# **_Tax-Loss Harvesting & Portfolio System_**

### **_About:_**
This notebook contains a portfolio system which, given a universe of stocks and a maximum size of N different stocks, the top N performing stocks are chosen to make up the portfolio and weighted according to a market cap scheme. A TLH engine is integrated into this system of screening and weighting. The TLH engine itself handles a single stock at a time but the portfolio system uses a loop to apply the engine to every asset.

**_Important Features & Points Of The Main Portfolio System:_**

* You can of course buy and sell assets on your portfolio which will then track those trades and the individual quantities of the assets held on the portfolio. Also it's important to note that the system doesn't support short selling but does allow for consecutive buys or sells at different dates and prices.

* The portfolio has a rebalancing capability as briefly mentioned above. First the top N stocks are screened and ranked using a monthly returns based system. Those N ranked stocks are then weighted using a market cap scheme, giving quantities to buy or sell for the N stocks.

* Any portfolio you make or have made can be saved and loaded to keep track of progress and trades you have made.

**_Important Features & Points Of The Tax-Loss Harvesting Engine:_**

* Currently the engine doesn't accurately tax the gains using a simplified long and short term rate of 10% and 20% respectively.

* The engine starts by checking if the trades of the given stock violate the wash sale rule with a transaction of the stock within the past 30 days.

* If the stock doesn't violate the wash sale rule, the engine proceeds to calculate the FIFO realised and unrealised gains and losses of that stock.

* Next the engine calculates the tax due before and after a harvest of the given stock. If the percentage savings of tax are greater than a given threshold, the engine returns true to signal that the given stock should be harvested.

### **_Structure and interactions between the classes:_**

![Image](Images/TLH%20object%20layout.jpg)

### **_Libraries_**

Every library necessary for this system to function.

In [166]:
# Data libraries
from bs4 import BeautifulSoup as bs
import yfinance as yf
import requests

# Data engineering
import pandas as pd
import numpy as np

# Time tools
from datetime import datetime, timedelta

# Saving
import pickle

### **_DataLoader_**

This object plays a key role in the entire system. It contains two methods. 

* The first method deals with live data, either the price or the market capital of the given asset which depends on the user's choice.

* The second method basically wraps around yfinance for the portfolio system to access past data of the chosen asset.

In [167]:
# DataLoader class
class DataLoader():
    
	# Initialiser
    def __init__(self):
        pass
    
	# Live data scraper
    def Live(self, symbol, choice):
        
		# Checking if price is chosen
        if choice == "price":
            
			# Setting variables to access the data
            headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36"}
            url = f"https://finance.yahoo.com/quote/{symbol}"
            
			# Extracting and parsing the HTML code
            request = requests.get(url, headers=headers)
            htmlcode = bs(request.text, "html.parser")
            
			# Finding and returning the price
            return float(htmlcode.find("fin-streamer", {"class": "livePrice yf-mgkamr"}).text.replace(",", ''))

		# Checking if market capitalisation is chosen
        elif choice == "mcap":

			# Setting variables to access the data
            headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36"}
            url = f"https://finance.yahoo.com/quote/{symbol}"

			# Extracting and parsing the HTML code
            request = requests.get(url, headers=headers)
            htmlcode = bs(request.text, "html.parser")

            # Extracting the market capitalisation expression
            cap = (htmlcode.find("fin-streamer", {"data-field": "marketCap"}).text.replace(" ", ''))    
            
			# Creating a dictionary of suffix multipliers
            multipliers = {
            	'K': 1000,
            	'M': 1000000,
        		'B': 1000000000,
        		'T': 1000000000000
            }
            
			# Checking if there is suffix on the extracted expression
            if cap[-1] in multipliers:
                
				# Extracting and using the multiplier
                multiplier = multipliers[cap[-1]]
                value = float(cap[:-1]) * multiplier
                return int(value)
            
			# Otherwise there's no suffix so returning the raw extraction
            else:
                
                # Converting the cap to a float
                return float(cap)
            
    # Past data scraper
    def Past(self, universe):

		# Download and extract the close data
        data = yf.download(universe, start=datetime.today()-timedelta(30), end=datetime.today())
        closes = data["Close"]
        
		# Returning the close prices
        return closes

### **_Stock Screener_**

This object isolates the top performing stocks in the given universe and is used during a regular rebalancing and TLH rebalancing of the portfolio.

* The screener object takes the past data of each asset in the universe of the portfolio and returns the ranked top N of the universe.

* This helps choose the optimum assets for the portfolio after rebalancing and if the given universe doesn't include the recently harvested assets, (if any) the wash sale rule is avoided.

In [168]:
# Stock screener class
class Screener():
    
	# Initialiser
    def __init__(self):
        pass
        
	# Method to actually screen the stocks
    def Screen(self, data, count):
        
		# Getting the bounds
        before=data.loc[datetime.today()-timedelta(30):datetime.today()].iloc[0]
        after=data.loc[datetime.today()-timedelta(30):datetime.today()].iloc[-1]
        
		# Calculating the monthly returns
        returns = (((after-before) / before) * 100).sort_values(ascending=False)
        
		# Filtering out the best performing
        return returns.iloc[:count]

### **_Allocator_**

This object performs the necessary calculations to allocate the ideal quantities of the screened stocks in the portfolio based on a market capital weighting scheme.

* Taking in the screened and ranked stocks, this object uses a premade DataLoader object to extract the live price and market cap for each stock.

* Next market capital proportions are calculated which are then used to assign the capital and those figures are then used with live prices to obtain the quantities.

In [169]:
# Allocator class
class Allocator():
    
	# Initialiser
    def __init__(self):
        pass
    
	# Method to allocate capital to stcoks according to Mcap weighting
    def Allocate(self, screened, capital, loader):
        
        # Creating lists for storage of prices and caps
        prices = []
        caps = []

		# Looping over the keys to extract prices for each
        for index in range(len(screened)):
            
			# Updating the lists
            prices.append(loader.Live(screened.keys()[index], "price"))
            caps.append(loader.Live(screened.keys()[index], "mcap"))

        # Converting the lists to arrays
        prices = np.array(prices)
        caps = np.array(caps)

		# Calculations for the respective quantities
        totalcap = sum(caps)
        proportions = caps / totalcap
        allocations = proportions * capital
        quantities = allocations // prices

		# Sorting everything into a dataframe
        details = pd.DataFrame({"Symbols":screened.keys(),
                                "Market Cap":caps,
                                "Proportions":proportions,
                                "Allocations":allocations,
                                "Quantities":quantities,
                                "Prices":prices})
        
		# Returning this dataframe to the user
        return details

### **_Gains Calculator_**

This object calculates the FIFO long-term and short-term, realised and unrealised, gains and or losses of an asset given it's trades.

* This is quite a complicated class which, in essence, is a large loop, first separating the long and short term trades.

* There are then separate processes which handle the long and short term gains in a FIFO manner, ultimately returning the necessary types of gains and losses.

In [170]:
# Calculating realised, unrealised, long and short term gains or losses
class Calculator():
    
	# Initialiser
    def __init__(self):
        pass
    
	# Method for calculating the gains or losses
    def Calculate(self, trades, current):
        
		# Calculating realised and unrealised, short and long term gains
        def CalculateGains(trades, current_price):
    
            # Initialise different gains storage
            realised_long_term_gains = 0
            realised_short_term_gains = 0
            unrealised_long_term_gains = 0
            unrealised_short_term_gains = 0

            # These lists contain their respective long positions
            all_long_positions = []
            short_term_long_positions = []
            long_term_long_positions = []

            # Looping over all trades
            for i in range(len(trades)):
    
                # Extracting a row for easier indexing
                row = trades.iloc[i]
    
                # Checking if it's a buy
                if row["Action"] == "buy":
    
                    # Appending all buys no matter the term
                    all_long_positions.append(row.copy())
    
                # Checking if it's a sell
                elif row["Action"] == "sell":
    
                    # Gathering specific sell details
                    og_shares_sold = row["Quantity"]
                    shares_sold = row["Quantity"]
                    price_sold = row["Price"]
                    date_sold = row["Date"]
                    total_short_term_cost_basis = 0
                    total_long_term_cost_basis = 0

                    # Reset positions lists
                    short_term_long_positions = []
                    long_term_long_positions = []

                    # Checking for long term or short term buys
                    for pos in all_long_positions:
    
                        # If the purchase was made more than a year before the sale
                        if pos["Date"] < (date_sold - timedelta(365)):
    
                            # Adding to the long term list
                            long_term_long_positions.append(pos)

                        # Otherwise                
                        else:
    
                            # Adding to the short term list
                            short_term_long_positions.append(pos)

                    # Process the shares sold for long term
                    while shares_sold > 0 and long_term_long_positions:
    
                        # Extracting the oldest long term buy
                        oldest_long_term_buy = long_term_long_positions[0]
    
                        # Checking if there are more shares sold than the quantity of the oldest buy
                        if shares_sold >= oldest_long_term_buy["Quantity"]:
    
                            # Calculating the cost basis
                            total_long_term_cost_basis += (oldest_long_term_buy["Quantity"] * oldest_long_term_buy["Price"])
    
                            # Updating the amount of shares left to handle
                            shares_sold -= oldest_long_term_buy["Quantity"]
    
                            # All oldest shares were used so they can be discarded
                            all_long_positions.remove(oldest_long_term_buy)
                            long_term_long_positions.pop(0)
    
                        # Otherwise deal with remaining shares
                        else:
    
                            # Calculating the basis as the remaining shares to sell times oldest price
                            total_long_term_cost_basis += (shares_sold * oldest_long_term_buy["Price"])
    
                            # Updating the amount of shares left on the oldest buy
                            oldest_long_term_buy["Quantity"] -= shares_sold

                            # No shares left to sell so setting it to zero which exits the loop
                            shares_sold = 0

                    # Total long term proceeds - the accurate cost basis
                    long_term_sold = og_shares_sold - shares_sold
                    long_term_proceeds = long_term_sold * price_sold
                    realised_long_term_gains += long_term_proceeds - total_long_term_cost_basis
    
                    # Reset shares sold for short-term calculation
                    shares_sold = og_shares_sold - long_term_sold

                    # Process the shares sold for short term
                    while shares_sold > 0 and short_term_long_positions:
    
                        # Extracting the oldest short term buy
                        oldest_short_term_buy = short_term_long_positions[0]
    
                        # Checking if there are more shares sold than the quantity of the oldest buy
                        if shares_sold >= oldest_short_term_buy["Quantity"]:
    
                            # Calculating the cost basis
                            total_short_term_cost_basis += (oldest_short_term_buy["Quantity"] * oldest_short_term_buy["Price"])
    
                            # Updating the amount of shares left to handle
                            shares_sold -= oldest_short_term_buy["Quantity"]
    
                            # All oldest shares were used so they can be discarded
                            all_long_positions.remove(oldest_short_term_buy)
                            short_term_long_positions.pop(0)
    
                        # Otherwise deal with remaining shares
                        else:
    
                            # Calculating the basis as the remaining shares to sell times oldest price
                            total_short_term_cost_basis += (shares_sold * oldest_short_term_buy["Price"])
    
                            # Updating the amount of shares left on the oldest buy
                            oldest_short_term_buy["Quantity"] -= shares_sold

                            # No shares left to sell so setting it to zero which exits the loop
                            shares_sold = 0

                    # Total short term proceeds - the accurate cost basis
                    short_term_sold = og_shares_sold - long_term_sold
                    short_term_proceeds = short_term_sold * price_sold
                    realised_short_term_gains += short_term_proceeds - total_short_term_cost_basis
            
            # Looping over the
            for position in all_long_positions:
        
		        # Calculating the current value of these positions
                current_value = position["Quantity"] * current_price
        
		        # Checking if the position is long or short term
                if position["Date"] < (datetime.today()-timedelta(365)):
            
			        # Updating the unrealised long term gains
                    unrealised_long_term_gains += current_value - (position["Quantity"] * position["Price"])
            
		        # Otherwise it's short term
                else:
            
			        # Updating the unrealised short term gains
                    unrealised_short_term_gains += current_value - (position["Quantity"] * position["Price"])
            
            # Returning the all the individual gains    
            return round(realised_long_term_gains, 2), round(realised_short_term_gains, 2), round(unrealised_long_term_gains, 2), round(unrealised_short_term_gains, 2)
		
		# Extracting the tuple of all gains
        all_gains = CalculateGains(trades, current)
        
		# Returning the gains to the user
        return all_gains       

### **_Wash Sale Checker_**

This object helps filter out assets who's trades violate the wash sale rule.

* The object's function is to return a boolean value, indicating whether the given asset is in violation of the wash sale rule.

* It deals with the time aspect of the wash sale rule by checking whether any of the trade's date falls within 30 days of current time. Other conditions are dealt elsewhere in the TLH engine and system.

In [171]:
# Checking trades for wash sale
class Checker():
    
	# Initialiser
    def __init__(Self):
        pass
    
	# Method for checking
    def Check(self, trades):
        
		# Setting time variables
        today = datetime.today()
        delta = timedelta(30)
        
		# Looping over the trades
        for i in range(len(trades)):
            
			# Isolating the trade date
            date = trades.iloc[i]["Date"]
            
            # Setting the status
            safe = True

			# Check if it falls within the period
            if (today-delta) <= date <= (today+delta):
                
				# Updating the status and exiting the loop
                safe = False
                break
                
		# Checking if the final status is safe
        if safe:
            
			# Returning true if they don't violate washsale
            return True
        
		# Otherwise...
        else:
            
			# Returning false as they don't violate washsale
            return False

### **_Calculate Tax_**

This object is an almost comprehensive taxer as it considers every situation including carry forward losses. It's only flaw is that it has simplified rates of tax with 10% for long-term gains and 20% for short-term gains.

* The object consists of a series of if and else statements which handle every single possible situation, returning the correct taxes and percentage savings (For the simplified rates).

* The object also has a feature to print out info which includes every type of the given gains, tax due before, tax due after and the individual savings.

In [172]:
# Calculating the tax due before and after a harvest
class Taxer():
    
    # Initialiser
    def __init__(self):
        pass
    
    # Method for calculating
    def Tax(self, gains, carry_forward, print_info):
        
        # Extracting the specific gains and setting some initial values
        long_term_realised, short_term_realised, long_term_unrealised, short_term_unrealised = gains
        long_term_remaining = long_term_realised
        short_term_remaining = short_term_realised
        final_forward = carry_forward
        og = carry_forward
        
		# Checking if the user wants printouts
        if print_info == True:
            
		    # Informing user of their inputs
            print(f"LT Realised Gains:   {long_term_realised}")
            print(f"ST Realised Gains:   {short_term_realised}")
            print(f"LT Unrealised Gains: {long_term_unrealised}")
            print(f"ST Unrealised Gains: {short_term_unrealised}")
        
        # Calculate initial tax before considering unrealized losses
        long_term_tax_before = max(0, long_term_realised) * 0.1
        short_term_tax_before = max(0, short_term_realised) * 0.2
        
		# Checking if the user wants printouts
        if print_info == True:
            
		    # Informing user of the tax due before a harvest
            print(f"LT before:           {round(long_term_tax_before, 2)}")
            print(f"ST before:           {round(short_term_tax_before, 2)}")
        
		# Handling long term unrealized losses
        if long_term_unrealised < 0 and long_term_realised > 0:
            
			# If the losses don't cancel the gains and more
            if long_term_realised >= abs(long_term_unrealised):
                
				# Calculating the tax and other important variables
                long_term_tax_after = (long_term_realised + long_term_unrealised) * 0.1
                long_term_remaining_loss = 0
                long_term_remaining = (long_term_realised + long_term_unrealised)
                # print("1. LT if")
                
			# Otherwise the losses completely cancel the gains and can be used elsewhere
            else:
                
				# Setting tax after to 0 and finding other important variables
                long_term_tax_after = 0
                long_term_remaining_loss = long_term_realised + long_term_unrealised
                long_term_remaining = 0
                # print("1. LT else")
                
		# No unrealised losses
        else:
            
			# Checking if gains are positive
            if long_term_unrealised + long_term_realised >= 0:
                
				# Calculating the tax after and other important variables
                long_term_tax_after = (long_term_realised + long_term_unrealised) * 0.1
                long_term_remaining_loss = 0
                long_term_remaining = (long_term_realised + long_term_unrealised)
                # print("2. LT if")
                
			# Otherwise there are leftover losses
            else:
                
				# Setting tax after to 0 and finding other important variables
                long_term_tax_after = 0
                long_term_remaining_loss = (long_term_realised + long_term_unrealised)
                long_term_remaining = 0
                # print("2. LT else")
                
        # Handling short term unrealised losses
        if short_term_unrealised < 0 and short_term_realised > 0:
            
			# If the losses don't cancel the gains and more
            if short_term_realised >= abs(short_term_unrealised):
                
				# Calculating the tax and other important variables
                short_term_tax_after = (short_term_realised + short_term_unrealised) * 0.2
                short_term_remaining_loss = 0
                short_term_remaining = (short_term_realised + short_term_unrealised)
                # print("3. ST if")
                
			# Otherwise the losses completely cancel out the gains and can be used elsewhere
            else:
                
				# Setting tax after to 0 and finding other important variables
                short_term_tax_after = 0
                short_term_remaining_loss = short_term_realised + short_term_unrealised
                short_term_remaining = 0
                # print("3. ST else")
                
		# No unrealised losses
        else:
            
            # Checking if gains are positive
            if short_term_unrealised + short_term_realised >= 0:
                
				# Calculating the tax after and other important variables
                short_term_tax_after = (short_term_realised + short_term_unrealised) * 0.2
                short_term_remaining_loss = 0
                short_term_remaining = (short_term_realised + short_term_unrealised)
                # print("4. ST if")
                
			# Otherwise there are leftover losses
            else:
                
				# Setting tax after to 0 and finding other important variables
                short_term_tax_after = 0
                short_term_remaining_loss = (short_term_realised + short_term_unrealised)
                short_term_remaining = 0
                # print("4. ST else")
            
		# Handling any remaining long term losses
        if long_term_remaining_loss < 0:
            
            # If there are still short term gains to offset
            if short_term_remaining > 0:

				# If the losses don't cancel the gains and more
                if short_term_remaining >= abs(long_term_remaining_loss):
                    
					# Calculating the new tax and finding other important variables
                    short_term_tax_after = (short_term_remaining + long_term_remaining_loss) * 0.2
                    short_term_remaining = (short_term_remaining + long_term_remaining_loss)
                    long_term_remaining_loss = 0
                    # print("5. if")
                
				# Otherwise there are still leftover losses
                else:
                    
					# Setting the remaining tax to 0 and setting other variables
                    short_term_tax_after = 0
                    long_term_remaining_loss = short_term_remaining + long_term_remaining_loss
                    short_term_remaining = 0
                    # print("5. else")

			# There are no more gains to offset
            else:
                # print("5. no gains")
                pass
                
		# Handling any remaining short term losses
        if short_term_remaining_loss < 0:
            
			# If there are still long term gains to offset
            if long_term_remaining > 0:
                
				# If the losses don't cancel the gains and more
                if long_term_remaining >= abs(short_term_remaining_loss):
                                        
					# Calculating the new tax and finding other important variables
                    long_term_tax_after = (long_term_remaining + short_term_remaining_loss) * 0.1
                    long_term_remaining = (long_term_remaining + short_term_remaining_loss)
                    short_term_remaining_loss = 0
                    # print("6. if")
                    
				# Otherwise there are still leftover losses
                else:
                    
					# Setting the remaining tax to 0 and setting other variables
                    long_term_tax_after = 0
                    short_term_remaining_loss = long_term_remaining + short_term_remaining_loss
                    long_term_remaining = 0
                    # print("6. else")
                    
			# There are no more gains to offset
            else:
                # print("6. no gains")
                pass
        
		# Checking if there are any carry forward losses
        if carry_forward < 0:
            
			# Checking if any short term gains left (offset them first because of the higher rate)
            if short_term_remaining > 0:
                
                # If the losses don't cancel the gains and more
                if short_term_remaining >= abs(carry_forward):
                    
                    # Calculating the new tax and setting other important variables
                    short_term_tax_after = (short_term_remaining + carry_forward) * 0.2
                    short_term_remaining = (short_term_remaining + carry_forward)
                    carry_forward = 0
                    # print("7. ST CF if")
                
				# Otherwise there are still leftover losses
                else:
                    
					# Setting the remaining tax to 0 and setting other variables
                    short_term_tax_after = 0
                    carry_forward = short_term_remaining + carry_forward
                    short_term_remaining = 0
                    # print("7. ST CF else")

			# There are no more gains to offset
            else:
                # print("7. ST CF no gains")
                pass

        # If there are still more carry forward
        if carry_forward < 0:
            
			# Checking if there are any long term gains to offset
            if long_term_remaining > 0:
                
				# If the losses don't cancel the gains and more
                if long_term_remaining >= abs(carry_forward):
                    
                    # Calculating the new tax and setting other important variables
                    long_term_tax_after = (long_term_remaining + carry_forward) * 0.1
                    long_term_remaining	= (long_term_remaining + carry_forward)
                    carry_forward = 0
                    # print("8. LT CF if")
                
				# Otherwise there are still leftover losses
                else:
                    
					# Setting the remaining tax to 0 ans setting other variables
                    long_term_tax_after = 0
                    carry_forward = long_term_remaining + carry_forward
                    long_term_remaining = 0
                    # print("8. LT CF else")
                    
			# There are no more gains to offset
            else:
                # print("8. LT CF no gains")
                pass

        # Calculating the amount of losses left to carry forward 
        finalcarry = sum(gains)
        finalcarry = finalcarry + final_forward
        
		# Calculating the savings used to determine worth of harvest
        long_term_savings = long_term_tax_before - long_term_tax_after
        short_term_savings = short_term_tax_before - short_term_tax_after
        total_tax_before = long_term_tax_before + short_term_tax_before
        total_tax_after = long_term_tax_after + short_term_tax_after
        total_tax_savings = total_tax_before - total_tax_after
        try:
            total_tax_percentage_savings = (round(total_tax_savings / total_tax_before, 2)) * 100
        except ZeroDivisionError as e:
            total_tax_percentage_savings = 0
            print(e)
            pass
        
		# Calculating the gains
        total_gains_before = long_term_realised + short_term_realised
        total_gains_after = total_gains_before + long_term_unrealised + short_term_unrealised + min(og, 0)
        net_gains_before = total_gains_before - (total_tax_before)
        net_gains_after = total_gains_after - (total_tax_after)
        
        # Checking if the user wants printouts
        if print_info == True:
            
		    # Printing final after taxes and any carry forward losses
            print(f"LT after:            {round(long_term_tax_after, 2)}")
            print(f"ST after:            {round(short_term_tax_after, 2)}")
            print(f"LT saved:            {round(long_term_savings, 2)}")
            print(f"ST saved:            {round(short_term_savings, 2)}")
            print(f"Total savings:       {round(total_tax_savings, 2)}")
            print(f"Total savings:       %{round(total_tax_percentage_savings, 2)}")
            print(f"Gross gains before:  {round(total_gains_before, 2)}")
            print(f"Gross gains after:   {round(total_gains_after, 2)}")
            print(f"Net gains before:    {round(net_gains_before, 2)}")
            print(f"Net gains after:     {round(net_gains_after, 2)}")
            print(f"Carry forward:       {min(finalcarry, 0)}")
            
		# Returning the saving percentage, list of taxes and carry forward losses
        return total_tax_percentage_savings, [long_term_tax_before, short_term_tax_before, long_term_tax_after, short_term_tax_after],  min(finalcarry, 0)

### **_TLH Engine_**

This object assembles some of the previous classes to function as the final TLH engine. It takes the trades of an individual asset and a threshold of the minimum precentage savings in tax which determines whether a True boolean value is returned, signaling the portfolio to harvest.

* After initialising all the necessary tools, the engine checks if the given trades violate the wash sale rule and if not, the engine continues.

* Next the engine calculates the gains and then the tax due on said gains. The percentage savings is then checked against the threshold and a boolean value is returned indicating whether the given asset should be haravested or not.

In [173]:
# Tax loss harvesting engine
class TaxLossHarvester():
    
	# Initialiser
    def __init__(self, checker, calculator, taxer, loader):
        
		# Setting the base functions for harvesting
        self.calculator = calculator
        self.checker = checker
        self.loader = loader
        self.taxer = taxer
    
	# Method for harvesting
    def Harvest(self, trades, threshold, carry, info):
        
		# Checking if the trades violate the wash sale
        status = self.checker.Check(trades)
        
		# Checking if the status is good
        if status == True:

            # Extracting the symbol and its price
            symbol = trades.iloc[0]["Symbol"]
            current = self.loader.Live(symbol, "price")

            # Extracting the realised and unrealised gains
            gains = self.calculator.Calculate(trades, current)
            
            # Calculating the taxes due
            savings, taxes, carry = self.taxer.Tax(gains, carry, info)

			# Checking conditions for the harvest
            if savings >= threshold:
                
				# It's good for harvesting
                return True, carry, savings, taxes

			# Otherwise no harvest
            else:

                # Not good for harvesting
                return False, carry, savings, taxes
            
		# Otherwise no harvest
        else:
            
			# Not good for harvesting
            return False, carry, 0, (0,0,0,0)

### **_Rebalancer_**

This object also takes some of the previous classes but contains methods for rebalancing the portfolio in multiple circumstances. The methods basically wrap around the basic allocator and screener objcects to tailor for the specific circumstances like initialisation and running the TLH engine.

* Each method first runs the basic objects in order to obtain the portfolio at the point in time of running. The methods then identify what needs to be changed in the current portfolio to achieve the ideal one that was just found for their specific case.

* The RebalanceInit method is of course for the initialisation of the portfolio. The ideal asset quantities given from the base objects are the same as the quantities needed to achieve the ideal portfolio so it's quite a simple method. 

* The RebalanceTLH method first narrows down the available universe and then checks if any ideal assets are in the portfolio already and updates the quantity to trade accordingly. 

* The RebalanceReg method first finds assets in the portfolio which are no longer in the ideal portfolio and negates their qauntity. Next it checks if any ideal assets are in the portfolio already and updates the quantity to trade accordingly.

In [174]:
# Class for rebalancing
class Rebalancer():
    
	# Initialiser
    def __init__(self, universe, capital, size, loader, screener, allocator):
        
		# Initialising the tools
        self.loader = loader
        self.screener = screener
        self.allocator = allocator
        
		# Setting portfolio attributes
        self.universe = universe.copy()
        self.capital = capital
        self.size = size
    
	# Method for initialising the portfolio's stocks
    def RebalanceInit(self):

		# Calculating the quantities for given stocks
        stocks = self.loader.Past(self.universe)
        screened = self.screener.Screen(stocks, self.size)
        allocated = self.allocator.Allocate(screened, self.capital, self.loader)
        specifics = allocated[["Symbols", "Quantities"]].copy()
        print("Goal quantities:")
        print(allocated)
        print()
        
		# Returning the allocated dataframe which contains quantities
        return specifics

	# Method for rebalancing based on TLH engine
    def RebalanceTLH(self, harvested, assets):
        
		# Setting the available universe to everything bar the harvested
        available = [asset for asset in self.universe if asset not in harvested]
        available_assets = assets.loc[[True if asset not in harvested else False for asset in assets["Symbol"].values]]
		
	    # Calculating the quantities for available
        stocks = self.loader.Past(available)
        screened = self.screener.Screen(stocks, self.size)
        allocated = self.allocator.Allocate(screened, self.capital, self.loader)
        specifics = allocated[["Symbols", "Quantities"]].copy()
        print("Goal quantities:")
        print(allocated)
        print()

        # Finding the stocks in assets that aren't in allocated
        for i in range(len(available_assets)):

			# Extracting the specifics of assets
            symbol = available_assets.iloc[i]["Symbol"]
            quantity = available_assets.iloc[i]["Quantity"]
            
		 	# Checking if it's not in allocated
            if symbol not in specifics["Symbols"].values:
                
		 	    # Updating the lists
                specifics.loc[len(specifics)+1] = [symbol, -quantity]
                    
			# Otherwise pass
            else:
                pass

		# Looping over the allocated
        for i in range(len(allocated)):
            
		 	# Extracting specifics
            symbol = allocated.iloc[i]["Symbols"]
            quantity = allocated.iloc[i]["Quantities"]
            
		 	# Checking if this asset is in the portfolio
            if symbol in assets["Symbol"].values:
                
                # Calculating the difference between the quantities
                difference = quantity - assets.loc[assets["Symbol"] == symbol]["Quantity"]
                specifics.loc[specifics["Symbols"]==symbol, "Quantities"] = difference.iloc[0]
        
        # Returning the allocated dataframe which contains quantities
        print("Necessary quantities:")
        print(specifics)
        print()
        return specifics
    
	# Method for a regular rebalance
    def RebalanceReg(self, assets):
        
		# Calculating the quantities for given stocks
        stocks = self.loader.Past(self.universe)
        screened = self.screener.Screen(stocks, self.size)
        allocated = self.allocator.Allocate(screened, self.capital, self.loader)
        specifics = allocated[["Symbols", "Quantities"]].copy()
        print("Goal quantities:")
        print(allocated)
        print()
            
		# Finding the stocks in assets that aren't in allocated
        for i in range(len(assets)):
            
            # Passing the filler value
            if assets.iloc[i]["Symbol"] != "filler":

			    # Extracting the specifics of assets
                symbol = assets.iloc[i]["Symbol"]
                quantity = assets.iloc[i]["Quantity"]

		 	    # Checking if it's not in allocated
                if symbol not in specifics["Symbols"].values:
                
		 		    # Updating the lists
                    specifics.loc[len(specifics)+1] = [symbol, -quantity]
                    
			# Otherwise pass
            else:
                pass
        
		# Finding the stocks in allocated that are in assets aready
        for i in range(len(allocated)):
            
		  	# Extracting the specifics of allocated
            symbol = allocated.iloc[i]["Symbols"]
            quantity = allocated.iloc[i]["Quantities"]
            
            # Checking if it's in assets
            if symbol in assets["Symbol"].values:
               
                # Calculating the difference in quantity between the two
                difference = quantity - assets.loc[assets["Symbol"] == symbol]["Quantity"]
                specifics.loc[specifics["Symbols"]==symbol, "Quantities"] = difference.iloc[0]
                
        # Returning the quantities to be traded
        print("Necessary quantities:")
        print(specifics)
        print()
        return specifics

### **_Order Manager_**

This object is crucial to the portfolio system as it creates a consistent bracket which is interpretable for the BuyAsset and SellAsset methods of the portfolio system.

* Taking in a DataLoader object, this object is able to gather live and accurate price data for the given trade.

* It returns an order bracket in the form of a pandas dataframe containing the necessary information for processing and logging the trade.

In [175]:
# Order manager class
class OrderManager():
    
	# Initialiser
    def __init__(self, loader):
        
		# Setting the dataloader
        self.loader = loader
    
	# Method to create an order
    def MakeOrder(self, symbol, quantity, side):
        
		# Creating a data loader and extracting the live price
        price = self.loader.Live(symbol, "price")
        
		# Getting today's date and time
        date = datetime.today()
        
		# Creating an order bracket
        bracket = pd.DataFrame({"Date":[date],
                                "Symbol":[symbol],
                                "Price":[price],
                                "Quantity":[quantity],
                                "Action":[side]})
        
		# Returning an order dataframe for the user
        return bracket

### **_Portfolio_**

The portfolio object allows for a collection of N different assets from a universe of size > N. Using its methods and the premade tools above, the portfolio can manage assets, track trades and monitor performance. Also the system can of course optimise your tax liablility with the TLH engine from above.

* Any portfolio object contains many attributes. These include the .universe which is a list of all available assets, .size which is the maximum number of different assets allowed in the portfolio, .capital which is the current funds available to purchase assets with, .trades which is a dataframe of every trade made on the portfolio, .assets which is a dataframe of all the assets currently being held on the portfolio, and finally all the attributes to manually access the tools like .loader and .manager.

* You can save your current portfolio or load a previously made one to keep track between sessions, you just have to provide a file name. This is achieved using pickling from the inbuilt python module.

* It's not reccomended but you can manually buy or sell assets for your portfolio using the respective methods. The reason why is because the methods are intended for the rebalancing and initialisation of the portfolio only and the manual operation doesn't account for wash sale and remaining capital.

* The Harvester method serves as a wrapper / interface between the TLH engine and the portfolio as it runs the engine assset by asset on the portfolio and then uses the TLH rebalance protocol.

* The Rebalance method works similarly as an interface between the regular rebalancing tool, first getting the quantities to trade to reach the ideal portfolio and then processes those with the buy and sell methods.

* The portfolio's performance can also be monitered using the CalculateGains method. It uses the calculator tool and returns the total realised and unrealised gains for the entire portfolio.

In [176]:
# Portfolio class
class Portfolio():
    
	# Initialiser, adds stocks
    def __init__(self, universe, capital, size, file=None):
        
        # If no file is provided, run regular initialisation protocol
        if file == None:

		    # Setting portfolio attributes
            self.carry = 0
            self.size = size
            self.capital = capital
            self.starting_capital = capital
            self.universe = universe.copy()
            self.available_universe = universe.copy()
        
		    # Setting manager dataframes for tracking trades and assets
            self.trades = pd.DataFrame({"Date":["filler"], "Symbol":["filler"], "Price":["filler"], "Quantity":["filler"], "Action":["filler"]})
            self.assets = pd.DataFrame({"Symbol":["filler"], "Quantity":["filler"]})

		    # Setting the tools of the portfolio
            self.taxer = Taxer()
            self.checker = Checker()
            self.calculator = Calculator()
            self.loader = DataLoader()
            self.screener = Screener()
            self.allocator = Allocator()
            self.manager = OrderManager(self.loader)
            self.engine = TaxLossHarvester(self.checker, self.calculator, self.taxer, self.loader)
            self.rebalancer = Rebalancer(self.universe, self.capital, self.size, self.loader, self.screener, self.allocator)

		    # Getting the quantities of stocks to initialise the portfolio
            allocated = self.rebalancer.RebalanceInit()
        
		    # Looping over the chosen stocks and adding them to the portfolio
            for index in range(len(allocated)):
            
			    # Adding the asset
                self.BuyAsset(allocated["Symbols"].iloc[index], allocated["Quantities"].iloc[index])
                
		# Otherwise attempt to load provided file data
        else:
            
			# Retrieving function
            self.LoadPortfolio(file)
            
			# Setting the tools of the portfolio
            self.taxer = Taxer()
            self.checker = Checker()
            self.calculator = Calculator()
            self.loader = DataLoader()
            self.screener = Screener()
            self.allocator = Allocator()
            self.manager = OrderManager(self.loader)
            self.engine = TaxLossHarvester(self.checker, self.calculator, self.taxer, self.loader)
            self.rebalancer = Rebalancer(self.universe, self.starting_capital, self.size, self.loader, self.screener, self.allocator)
        
	# Method to buy an asset for the portfolio
    def BuyAsset(self, symbol, quantity):
        
		# Creating the order
        order = self.manager.MakeOrder(symbol, quantity, "buy")
        
		# If the asset isn't already present on the portfolio
        if order["Symbol"][0] not in self.assets["Symbol"].values:
            
			# Checking if a new stock breaches the portfolio size
            if (len(self.assets)-1) >= self.size:
                print("Portfolio size limit reached. Cannot add new asset")
                print(f"Symbol: {order['Symbol'][0]}, Quantity: {order['Quantity'][0]}")
                return
            
			# Adding the asset to the portfolio asset list
            update = pd.DataFrame({"Symbol":[order["Symbol"][0]], "Quantity":[order["Quantity"].values[0]]})
            self.assets = pd.concat([self.assets, update]).reset_index(drop=True)
            
			# Logging the trade
            self.trades = pd.concat([self.trades, order]).reset_index(drop=True)
            
			# Updating the list of available stocks that aren't under the portfolio
            self.available_universe.remove(order["Symbol"][0])
        
		# Otherwise if the asset is already in the portfolio
        else:
            
			# Updating the quantity of the assets
            current_quantity = self.assets.loc[self.assets["Symbol"]==order["Symbol"][0], "Quantity"].values[0]
            new_quantity = current_quantity + order["Quantity"].values[0]
            self.assets.loc[self.assets["Symbol"]==order["Symbol"][0], "Quantity"] = new_quantity
            
			# Logging the other trades
            self.trades = pd.concat([self.trades, order]).reset_index(drop=True)

		# Updating the capital
        self.capital -= float((order["Quantity"] * order["Price"]).iloc[0])
    
	# Method to sell an asset for the portfolio
    def SellAsset(self, symbol, quantity):
        
        # Creating the order
        order = self.manager.MakeOrder(symbol, quantity, "sell")

		# If the asset is already present on the portfolio
        if order["Symbol"][0] in self.assets["Symbol"].values:
            
			# Calculating the new portfolio quantity of the asset
            current_quantity = self.assets.loc[self.assets["Symbol"]==order["Symbol"][0], "Quantity"].values[0]
            new_quantity = current_quantity - order["Quantity"].values[0]
            
			# Checking if the sale quantity was greater than current quantity (Short sale)
            if new_quantity < 0:
                
				# Inform the user and exit
                print("Quantity too great, no short sales allowed")
                print(f"Symbol: {order['Symbol'][0]}, Quantity: {order['Quantity'][0]}")
            
			# If the sale cancels out position completely then remove it from assets
            elif new_quantity == 0:
                
				# Removing the asset from the portfolio
                self.assets = self.assets.loc[self.assets["Symbol"] != order["Symbol"][0]].reset_index(drop=True)
                
				# Logging the trade
                self.trades = pd.concat([self.trades, order]).reset_index(drop=True)
                
				# Updating the capital
                self.capital += (order["Quantity"] * order["Price"])
                
				# Updating the stocks that we don't have under the portfolio
                self.available_universe.append(order["Symbol"][0])
            
			# Otherwise if there would still be a remaining amount
            else:
                
				# Updating the asset's quantity on the portfolio
                self.assets.loc[self.assets["Symbol"]==order["Symbol"][0], "Quantity"] = new_quantity
                
				# Combining with the other trades
                self.trades = pd.concat([self.trades, order]).reset_index(drop=True)
                
				# Updating the capital
                self.capital += float((order["Quantity"] * order["Price"]).iloc[0])
            
		# Otherwise the position isn't present and shorting isn't allowed so...
        else:
            
			# Informing the user
            print("This position is not present and shorting isn't available")
            print(f"Symbol: {order['Symbol'][0]}, Quantity: {order['Quantity'][0]}")
    
	# Method to run the TLH engine
    def Harvester(self, threshold, taxinfo):
		
        # Setting a copy of the assets
        temp_assets = self.assets.copy()
        harvests = []
        
		# Looping over all assets to assess quality of harvest
        for i in range(len(temp_assets)):
            
			# Extracting the individual asset and information
            current_asset = temp_assets.iloc[i]["Symbol"]
            current_quantity = temp_assets.iloc[i]["Quantity"]
            current_trades = self.trades.loc[self.trades["Symbol"] == current_asset]
            
            # Passing the filler value
            if current_asset != "filler":

			    # Checking if a harvest is allowed and good
                status, self.carry, savings, taxes = self.engine.Harvest(current_trades, threshold, self.carry, taxinfo)
                print(f"Asset:                {current_asset:<16} Harvest:               {status}")
                print(f"Savings:              %{savings:<15} Carry forward losses:  ${self.carry:<15.2f}")
                print(f"Long term tax before: ${taxes[0]:<15.2f} Short term tax before: ${taxes[1]:<15.2f}")
                print(f"Long term tax after:  ${taxes[2]:<15.2f} Short term tax after:  ${taxes[3]:<15.2f}")
                print()
                if status == True:
					
				    # Harvesting
                    self.SellAsset(current_asset, current_quantity)
                    harvests.append(current_asset)

        # Getting the quantities to trade for the rebalance
        allocated = self.rebalancer.RebalanceTLH(harvests, temp_assets)
        
		# Sorting allocations by quant ascending to avoid full portfolio issue
        allocated = allocated.sort_values(by="Quantities").reset_index(drop=True)
        
		# Looping over the allocated and checking if we need to buy or sell
        for index in range(len(allocated)):
            
			# Checking for a buy
            if allocated["Quantities"].iloc[index] > 0:
                
				# Buying the given quantity of given asset
                self.BuyAsset(allocated["Symbols"].iloc[index], allocated["Quantities"].iloc[index])
                
			# Checking for a sell
            elif allocated["Quantities"].iloc[index] < 0:
               
				# Selling the given quantity of given asset
                self.SellAsset(allocated["Symbols"].iloc[index], abs(allocated["Quantities"].iloc[index]))
                
			# Otherwise the quantity is 0
            else:
                pass
            
	# Method to rebalance the portfolio
    def Rebalance(self):
        
		# Getting the quantities to trade for the rebalance
        allocated = self.rebalancer.RebalanceReg(self.assets)
        
		# Sorting allocations by quant ascending to avoid full portfolio issue
        allocated = allocated.sort_values(by="Quantities").reset_index(drop=True)

		# Looping over the allocated and checking if we need to buy or sell
        for index in range(len(allocated)):
            
			# Checking for a buy
            if allocated["Quantities"].iloc[index] > 0:
                
				# Buying the given quantity of given asset
                self.BuyAsset(allocated["Symbols"].iloc[index], allocated["Quantities"].iloc[index])
                
			# Checking for a sell
            elif allocated["Quantities"].iloc[index] < 0:
                
				# Selling the given quantity of given asset
                self.SellAsset(allocated["Symbols"].iloc[index], abs(allocated["Quantities"].iloc[index]))
                
			# Otherwise the quantity is 0
            else:
                pass
            
    # Method to calculate gains
    def CurrentGains(self):
        
        # Storage of gains
        total_realised = 0
        total_unrealised = 0
        
		# Isolating every asset to ever be traded on the portfolio
        assets_w_dups = []
        
		# Looping over the trades
        for i in range(len(self.trades)):
            
            # Passing the filler value
            if self.trades.iloc[i]["Symbol"] != "filler":
                
                # Updating the list
                assets_w_dups.append(self.trades.iloc[i]["Symbol"])
                
		    # Passing otherwise
            else:
                pass
            
		# Removing duplicates by creating a set
        assets = set(assets_w_dups)
        assets = list(assets) 

		# Looping over the every asset ever
        for i in range(len(assets)):
                
			# Calculating the individual gains and updating the totals
            gains = self.calculator.Calculate(self.trades.loc[self.trades["Symbol"]==assets[i]], self.loader.Live(assets[i], "price"))
            total_realised += (gains[0] + gains[1])
            total_unrealised += (gains[2] + gains[3])

		# Informing the user and returning the gains
        print(f"Total Realised Gains:   {round(total_realised, 2)}")
        print(f"Total Unrealised Gains: {round(total_unrealised, 2)}")
        return total_realised, total_unrealised

	# Method to save current portfolio data
    def SavePortfolio(self, filename):
        
        # Extracting the attributes
        data = {"carry":self.carry,
                "size":self.size,
                "capital":self.capital,
                "starting_capital":self.starting_capital,
                "universe":self.universe,
                "available_universe":self.available_universe,
                "trades":self.trades,
                "assets":self.assets}

		# Context manager
        with open(filename, "wb") as f:
            pickle.dump(data, f)
            
	# Method for loading previous portfolio data
    def LoadPortfolio(self, filename):
        
		# Attempting to retrieve data
        try:
            
			# Context manager
            with open(filename, "rb") as f:
                
				# Loading the data and pasting it to the portfolio
                data = pickle.load(f)
                self.carry = data["carry"]
                self.size = data["size"]
                self.capital = data["capital"]
                self.starting_capital = data["starting_capital"]
                self.universe = data["universe"]
                self.available_universe = data["available_universe"]
                self.trades = data["trades"]
                self.assets = data["assets"]
        
		# Handling a file not found error
        except FileNotFoundError:
            print("Save file not found.")

## **_Example 1: Initialisation_**

In this first cell a portfolio object is initialised from scratch. This recquires a selection of assets for your universe, a figure for your starting capital and a set size of the portfolio (The file is set to None because we're initialising from scratch). Here's what happens when we initialise:

* Since file == None, the parameters are set as attributes and the tools are also initialised as attributes.

* Next the rebalancer tool calls its RebalanceInit method which allocates the quantities of the best assets according to the market cap weighting scheme.

In [177]:
# Setting a universe and other parameters of the portfolio
universe = ["AAPL", "AMZN", "MSFT", "META", "NFLX", "NVDA", "GOOG", "TSLA", "COST", "WMT", "LLY"]
capital = 100000
size = 5
file = None

# Initialising
Example1Pf = Portfolio(universe, capital, size, file)

[*********************100%%**********************]  11 of 11 completed


Goal quantities:
  Symbols     Market Cap  Proportions   Allocations  Quantities  Prices
0     WMT   555527000000     0.073797   7379.695227       106.0   69.07
1    AAPL  3381000000000     0.449137  44913.657777       203.0  220.48
2    META  1284000000000     0.170568  17056.828331        33.0  506.33
3    AMZN  1947000000000     0.258642  25864.209314       138.0  187.11
4    COST   360250000000     0.047856   4785.609351         5.0  812.59



This cell simply prints out the various attributes that contain info on the portfolio.

* The universe attribute doesn't change no matter the action on the portfolio. It of course contains a list of all the assets the portfolio can choose from.

* The capital attribute does change through use of the portfolio and is handled in the BuyAsset and SellAsset methods. As you can see it's far less than the starting capital.

* The starting capital attribute doesn't change through use and is exactly what it says on the tin, the amount of money the portfolio was given at initialisation.

In [178]:
# Info attributes
print(f"Universe of assets:             {Example1Pf.universe}")
print(f"Remaining capital:              {round(Example1Pf.capital, 2)}")
print(f"Starting capital:               {Example1Pf.starting_capital}")
print(f"Max size of portfolio:          {Example1Pf.size}")

Universe of assets:             ['AAPL', 'AMZN', 'MSFT', 'META', 'NFLX', 'NVDA', 'GOOG', 'TSLA', 'COST', 'WMT', 'LLY']
Remaining capital:              1282.25
Starting capital:               100000
Max size of portfolio:          5


This cell demonstrates the tracking features of the portfolio using the unreccomended manual sale of one of the assets.

* The trades and assets attributes are called and printed out first for comparison after we make the manual sale.

* Next the SellAsset method is called which creates an order bracket using the OrderManager tool. Depending on the effect of this order on the portfolio, its adjusted accordingly.

* The trades and assets attributes are called once again showing the effect on each dataframe.

In [179]:
# Trades dataframe
print("Trades dataframe:")
print(Example1Pf.trades.iloc[1:])
print()

# Assets dataframe
print("Assets dataframe:")
print(Example1Pf.assets.iloc[1:])
print()

# Not reccomended to make manual trades!
Example1Pf.SellAsset("AAPL", 15)

# Trades dataframe
print("Traded dataframe post manual trade:")
print(Example1Pf.trades.iloc[1:])
print()

# Assets dataframe
print("Assets dataframe post manual trade:")
print(Example1Pf.assets.iloc[1:])
print()

Trades dataframe:
                         Date Symbol   Price Quantity Action
1  2024-08-01 16:21:00.962726    WMT   69.07    106.0    buy
2  2024-08-01 16:21:03.265912   AAPL  220.51    203.0    buy
3  2024-08-01 16:21:04.317577   META  506.95     33.0    buy
4  2024-08-01 16:21:05.192879   AMZN  187.25    138.0    buy
5  2024-08-01 16:21:06.165128   COST  812.59      5.0    buy

Assets dataframe:
  Symbol Quantity
1    WMT    106.0
2   AAPL    203.0
3   META     33.0
4   AMZN    138.0
5   COST      5.0

Traded dataframe post manual trade:
                         Date Symbol   Price Quantity Action
1  2024-08-01 16:21:00.962726    WMT   69.07    106.0    buy
2  2024-08-01 16:21:03.265912   AAPL  220.51    203.0    buy
3  2024-08-01 16:21:04.317577   META  506.95     33.0    buy
4  2024-08-01 16:21:05.192879   AMZN  187.25    138.0    buy
5  2024-08-01 16:21:06.165128   COST  812.59      5.0    buy
6  2024-08-01 16:21:28.169396   AAPL  220.31       15   sell

Assets dataframe post ma

This cell shows the manual use of some of the tool attributes. These don't affect the portfolio they originate from.

* First the loader object is called and returns the two types of live data available, the price and the market capital of the given symbol.

* An order bracket is created manually but there isn't really any use for doing this because it's done in the BuyAsset and SellAsset methods.

* The screener tool is used and shows the assets of the universe with the top monthly returns.

In [183]:
# Tool attribute examples
print(f"Live AAPL price: {Example1Pf.loader.Live('AAPL', 'price')}")
print(f"Live AAPL mcap: {Example1Pf.loader.Live('AAPL', 'mcap')}")
print()
print(f"Manual order bracket:")
print(Example1Pf.manager.MakeOrder('AAPL', 10, 'buy'))
print()
print(f"Screening the top 5 of the universe: ")
print(Example1Pf.screener.Screen(Example1Pf.loader.Past(Example1Pf.universe), 5))

Live AAPL price: 220.31
Live AAPL mcap: 3378000000000

Manual order bracket:
                        Date Symbol   Price  Quantity Action
0 2024-08-01 16:23:02.399102   AAPL  220.29        10    buy



[*********************100%%**********************]  11 of 11 completed

Screening the top 5 of the universe: 
Ticker
WMT     1.362838
AAPL   -0.568725
META   -0.911249
AMZN   -5.430435
COST   -5.767046
dtype: float64


This cell demonstrates the the saving feature of the portfolio. It saves the key attributes like assets, trades and more.

* A filename with the .pkl suffix for the pickle library must be provided

* First a data dictionary is created with the necessary attributes

* The dictionary is then pickled and stored under the given filename

In [184]:
# Saving the portfolio for the next session
Example1Pf.SavePortfolio("Example1Pf.pkl")

## **_Example 2: Rebalancing_**

In this first cell a portfolio object is initialised from a presaved file. This just recquires the name of the file in which the portfolio details are located (The other parameters are set to None because they are stored in the file). Here's what happens when we initialise from a file:

* Since file != None, the LoadPortfolio method is called which sets the most of the attributes like capital, trades and assets. 

* The rest of the tool attributes are initialised afterwards because they use some of the saved attributes.

In [185]:
# Initialising from a premade portfolio
Example2Pf = Portfolio(None, None, None, "Example2Pf.pkl")

This cell simply prints out the various attributes that contain info on the portfolio.

* The universe attribute doesn't change no matter the action on the portfolio. It of course contains a list of all the assets the portfolio can choose from.

* The capital attribute does change through use of the portfolio and is handled in the BuyAsset and SellAsset methods. As you can see it's far less than the starting capital.

* The starting capital attribute doesn't change through use and is exactly what it says on the tin, the amount of money the portfolio was given at initialisation.

In [186]:
# Info attributes
print(f"Universe of assets:     {Example2Pf.universe}")
print(f"Remaining capital:      {round(Example2Pf.capital, 2)}")
print(f"Starting capital:       {Example2Pf.starting_capital}")
print(f"Max size of portfolio:  {Example2Pf.size}")

Universe of assets:     ['AAPL', 'AMZN', 'MSFT', 'META', 'TSLA', 'GOOG', 'NVDA', 'NFLX']
Remaining capital:      477.73
Starting capital:       10000
Max size of portfolio:  3


This cell simply shows the trades and assets dataframe before the rebalance for a reference later on

* The trades dataframe is printed out and many past trades can be seen, some recent and some from a while ago.

* The assets dataframe is also printed and the current assets held on the portfolio after all of the trades can be seen.

In [187]:
# Trades dataframe
print("Trades dataframe:")
print(Example2Pf.trades)
print()

# Assets dataframe
print("Assets dataframe:")
print(Example2Pf.assets)
print()

Trades dataframe:
         Date Symbol  Price  Quantity Action
0  2020-01-01   AAPL    200        17    buy
1  2020-01-02   AMZN    270        10    buy
2  2020-01-03   MSFT    280        10    buy
3  2020-06-01   AAPL    250         4   sell
4  2020-06-02   AMZN    280         3   sell
5  2020-06-03   MSFT    320         4   sell
6  2021-01-01   AAPL    230         4   sell
7  2021-01-02   AMZN    300         3   sell
8  2021-01-03   MSFT    360         4   sell
9  2024-06-01   AAPL    240         8    buy
10 2024-06-02   AMZN    200         8    buy
11 2024-06-03   MSFT    370         6    buy

Assets dataframe:
  Symbol  Quantity
0   AAPL        17
1   MSFT         8
2   AMZN        12



This cell shows the current gains of this portfolio using the CurrentGains method which is basically a wrapper for the Calculator tool which is initialised at the beginning of the portfolio.

* As the Calculator tool handles data on a single asset basis, the method loops over the entire portfolio's asset's trades and updates the total on each iteration.

* Finally the totals for the realised and unrealised gains of the portfolio are printed out.

In [188]:
# Extracting and printing the gains
realised, unrealised = Example2Pf.CurrentGains()

Total Realised Gains:   920
Total Unrealised Gains: 163.49


This cell is where the rebalancing takes place. The portfolio reconsiders all of its options and decides what the new best combination of assets in the given universe is.

* The first dataframe printed out contains the ideal quantities for the new best portfolio (The quantities we want to get to). These figures are generated in the first half of the RebalanceReg method of the rebalancer tool.

* The second dataframe contains trhe quantities which need to be traded to achieve the ideal quantities in the previous dataframe. The second half of the RebalanceReg method of the rebalancer tool creates this dataframe by checking for both assets that were in the portfolio previously and still are but also assets that are no longer in the ideal portfolio at all.

* The rest of the Rebalance method of the portfolio processes the quantites of the second dataframe to match the ideal quantites of the first dataframe to the quantities of the current assets in the portfolio.

In [189]:
# Rebalancing
Example2Pf.Rebalance()

[*********************100%%**********************]  8 of 8 completed


Goal quantities:
  Symbols     Market Cap  Proportions  Allocations  Quantities  Prices
0    AAPL  3377000000000     0.511589  5115.891532        23.0  220.21
1    META  1280000000000     0.193910  1939.100136         3.0  504.44
2    AMZN  1944000000000     0.294501  2945.008332        15.0  186.83

Necessary quantities:
  Symbols  Quantities
0    AAPL         6.0
1    META         3.0
2    AMZN         3.0
4    MSFT        -8.0



Here, the new trades that were made to achieve the ideal portfolio are printed. So are the current assets which should match the ideal qunatities.

* The trades dataframe now contains trades with the same quantities as the quantities in the second dataframe of the previous cell.

* The assets dataframe now contains the ideal assets with the correct quantites as seen in the first dataframe of the previous cell.

In [190]:
# Trades dataframe
print("Trades dataframe:")
print(Example2Pf.trades)
print()

# Assets dataframe
print("Assets dataframe:")
print(Example2Pf.assets)
print()

Trades dataframe:
                         Date Symbol   Price  Quantity Action
0  2020-01-01 00:00:00.000000   AAPL  200.00      17.0    buy
1  2020-01-02 00:00:00.000000   AMZN  270.00      10.0    buy
2  2020-01-03 00:00:00.000000   MSFT  280.00      10.0    buy
3  2020-06-01 00:00:00.000000   AAPL  250.00       4.0   sell
4  2020-06-02 00:00:00.000000   AMZN  280.00       3.0   sell
5  2020-06-03 00:00:00.000000   MSFT  320.00       4.0   sell
6  2021-01-01 00:00:00.000000   AAPL  230.00       4.0   sell
7  2021-01-02 00:00:00.000000   AMZN  300.00       3.0   sell
8  2021-01-03 00:00:00.000000   MSFT  360.00       4.0   sell
9  2024-06-01 00:00:00.000000   AAPL  240.00       8.0    buy
10 2024-06-02 00:00:00.000000   AMZN  200.00       8.0    buy
11 2024-06-03 00:00:00.000000   MSFT  370.00       6.0    buy
12 2024-08-01 16:23:36.727852   MSFT  419.74       8.0   sell
13 2024-08-01 16:23:37.546127   META  504.71       3.0    buy
14 2024-08-01 16:23:38.404320   AMZN  186.82       3

Here's the gains after the rebalance of the portfolio.

* Looking at the realised gains, you can see that we've gained roughly a hundred from the rebalance because we completely replaced the MSFT and AMZN positions.

* The unrealised gains lowers as we've realised the MSFT and AMZN unrealised gains and now we really just have the unrealised gains of the AAPL position left.

In [191]:
# Extracting and printing the gains
realised, unrealised = Example2Pf.CurrentGains()

Total Realised Gains:   1497.92
Total Unrealised Gains: -414.78


Also here's the new remaining capital we have. 

In [193]:
# New remaining capital
print(f"Remaining capital: {round(Example2Pf.capital[0], 2)}")

Remaining capital: 439.38


## **_Example 3: Harvesting_**

In this first cell a portfolio object is initialised from a presaved file. This just recquires the name of the file in which the portfolio details are located (The other parameters are set to None because they are stored in the file). Here's what happens when we initialise from a file:

* Since file != None, the LoadPortfolio method is called which sets the most of the attributes like capital, trades and assets. 

* The rest of the tool attributes are initialised afterwards because they use some of the saved attributes.

In [194]:
# Initialising from a premade portfolio
Example3Pf = Portfolio(None, None, None, "Example3Pf.pkl")

This cell simply prints out the various attributes that contain info on the portfolio.

* The universe attribute doesn't change no matter the action on the portfolio. It of course contains a list of all the assets the portfolio can choose from.

* The capital attribute does change through use of the portfolio and is handled in the BuyAsset and SellAsset methods. As you can see it's far less than the starting capital.

* The starting capital attribute doesn't change through use and is exactly what it says on the tin, the amount of money the portfolio was given at initialisation.

In [195]:
# Info attributes
print(f"Universe of assets:     {Example3Pf.universe}")
print(f"Remaining capital:      {round(Example3Pf.capital, 2)}")
print(f"Starting capital:       {Example3Pf.starting_capital}")
print(f"Max size of portfolio:  {Example3Pf.size}")

Universe of assets:     ['AAPL', 'AMZN', 'MSFT', 'META', 'TSLA', 'GOOG', 'NVDA', 'NFLX']
Remaining capital:      477.63
Starting capital:       10000
Max size of portfolio:  3


This cell simply shows the trades and assets dataframe before the harvest for a reference later on

* The trades dataframe is printed out and many past trades can be seen, some recent and some from a while ago.

* The assets dataframe is also printed and the current assets held on the portfolio after all of the trades can be seen.

In [196]:
# Trades dataframe
print("Trades dataframe")
print(Example3Pf.trades)
print()

# Assets dataframe
print("Assets dataframe")
print(Example3Pf.assets)
print()

Trades dataframe
         Date Symbol  Price  Quantity Action
0  2020-01-01   AAPL  230.0      17.0    buy
1  2020-01-02   AMZN  270.0      10.0    buy
2  2020-01-03   MSFT  320.0      10.0    buy
3  2020-06-01   AAPL  250.0       4.0   sell
4  2020-06-02   AMZN  280.0       3.0   sell
5  2020-06-03   MSFT  320.0       4.0   sell
6  2021-01-01   AAPL  230.0       4.0   sell
7  2021-01-02   AMZN  300.0       3.0   sell
8  2021-01-03   MSFT  360.0       4.0   sell
9  2024-06-01   AAPL  240.0       8.0    buy
10 2024-06-02   AMZN  200.0       8.0    buy
11 2024-06-03   MSFT  370.0       6.0    buy

Assets dataframe
  Symbol  Quantity
0   AAPL        17
1   MSFT         8
2   AMZN        12



This cell shows the current gains of this portfolio using the CurrentGains method which is basically a wrapper for the Calculator tool which is initialised at the beginning of the portfolio.

* As the Calculator tool handles data on a single asset basis, the method loops over the entire portfolio's asset's trades and updates the total on each iteration.

* Finally the totals for the realised and unrealised gains of the portfolio are printed out.

In [197]:
# Extracting and printing the gains
realised, unrealised = Example3Pf.CurrentGains()

Total Realised Gains:   360.0
Total Unrealised Gains: -176.67


This cell is where the harvesting takes place. The portfolio identifies the best options (if any) for harvesting and then the portfolio rebalances using the market cap weighting scheme after the harvest.

* First, the assets currently held on the portfolio are looped over and the base engine analyses it. First checking if the trades of the given asset violate the wash sale rule, if they don't, the gains and then tax due on those gains before and after a harvest are calculated. If the savings from a harvest are better than the threshold, then the asset is harvested. The status on whether the asset was harvested or not is printed out too.

* The First dataframe printed out shows the new ideal portfolio excluding the harvested assets should they be in the top ranked. These figures are obtained from the RebalanceTLH method of the rebalancer tool which like I said, finds the ideal portfolio according to the market cap weighting scheme while excluding the harvested assets.

* The second dataframe contains the quantities which need to be traded to achieve the ideal quantities in the previous dataframe. The second half of the RebalanceTLH method of the rebalancer tool creates this dataframe by checking for both assets that were in the portfolio previously and still are but also assets that are no longer in the ideal portfolio at all (of course excluding the harvested stocks).

* The rest of the Rebalance method of the portfolio processes the quantites of the second dataframe to match the ideal quantites of the first dataframe to the quantities of the current assets in the portfolio.

In [198]:
# Setting a threshold and a printing preference
threshold = 20 
taxinfo = False

# Harvesting and rebalancing the portfolio
Example3Pf.Harvester(threshold, taxinfo)

Asset:                AAPL             Harvest:               True
Savings:              %100.0           Carry forward losses:  $-158.10        
Long term tax before: $0.00            Short term tax before: $16.00          
Long term tax after:  $0.00            Short term tax after:  $0.00           

Asset:                MSFT             Harvest:               False
Savings:              %-303.0          Carry forward losses:  $0.00           
Long term tax before: $16.00           Short term tax before: $0.00           
Long term tax after:  $36.00           Short term tax after:  $28.40          

Asset:                AMZN             Harvest:               True
Savings:              %100.0           Carry forward losses:  $-320.20        
Long term tax before: $9.00            Short term tax before: $6.00           
Long term tax after:  $0.00            Short term tax after:  $0.00           



[*********************100%%**********************]  6 of 6 completed


Goal quantities:
  Symbols     Market Cap  Proportions  Allocations  Quantities  Prices
0    META  1283000000000     0.349049  3490.491607         6.0  505.69
1    NFLX   272700000000     0.074190   741.899502         1.0  635.42
2    GOOG  2120000000000     0.576761  5767.608891        33.0  173.07

Necessary quantities:
  Symbols  Quantities
0    META         6.0
1    NFLX         1.0
2    GOOG        33.0
4    MSFT        -8.0



Here, the new trades that were made to achieve the harvest and ideal portfolio are printed. So are the current assets which should match the ideal qunatities.

* The trades dataframe now contains trades with the same quantities as the quantities in the second dataframe of the previous cell.

* The assets dataframe now contains the ideal assets with the correct quantites as seen in the first dataframe of the previous cell.

In [199]:
# Trades dataframe
print("Trades dataframe")
print(Example3Pf.trades)
print()

# Assets dataframe
print("Assets dataframe")
print(Example3Pf.assets)
print()

Trades dataframe
                         Date Symbol   Price  Quantity Action
0  2020-01-01 00:00:00.000000   AAPL  230.00      17.0    buy
1  2020-01-02 00:00:00.000000   AMZN  270.00      10.0    buy
2  2020-01-03 00:00:00.000000   MSFT  320.00      10.0    buy
3  2020-06-01 00:00:00.000000   AAPL  250.00       4.0   sell
4  2020-06-02 00:00:00.000000   AMZN  280.00       3.0   sell
5  2020-06-03 00:00:00.000000   MSFT  320.00       4.0   sell
6  2021-01-01 00:00:00.000000   AAPL  230.00       4.0   sell
7  2021-01-02 00:00:00.000000   AMZN  300.00       3.0   sell
8  2021-01-03 00:00:00.000000   MSFT  360.00       4.0   sell
9  2024-06-01 00:00:00.000000   AAPL  240.00       8.0    buy
10 2024-06-02 00:00:00.000000   AMZN  200.00       8.0    buy
11 2024-06-03 00:00:00.000000   MSFT  370.00       6.0    buy
12 2024-08-01 16:25:58.488223   AAPL  220.71      17.0   sell
13 2024-08-01 16:26:01.316666   AMZN  186.65      12.0   sell
14 2024-08-01 16:26:08.074534   MSFT  420.13       8.

Here's the gains after the harvest and rebalance of the portfolio.

* Looking at the realised gains we can see we've lost around 160 because we harvested 2 positions that were at a loss. Despite losing realised gains, we saved on tax and removed a position that could've potentially lost even more if we didn't deal with it in time.

* The unrealised gains is near 0 because we've ended up swapping out the portfolio completely.

In [201]:
# Extracting and printing the gains
realised, unrealised = Example3Pf.CurrentGains()

Total Realised Gains:   182.91
Total Unrealised Gains: -0.46


Also, here's the remaining capital we have.

In [202]:
# New remaining capital
print(f"Remaining capital: {round(Example3Pf.capital[0], 2)}")

Remaining capital: 451.83


Finally here's the remaining carry forward loss if there is any

In [203]:
# Carry forward losses from after the harvest
print(f"Carry forward losses: {round(Example3Pf.carry, 2)}")

Carry forward losses: -320.2
